In [48]:
import pandas as pd
import numpy as np
import requests
import io

url = 'https://nl.wikipedia.org/wiki/Lijst_van_landen_naar_inwonertal'

proxy_dict = {
    'http': 'proxy.ams.intra.schiphol.nl:8080',
    'https': 'proxy.ams.intra.schiphol.nl:8080'}

s = requests.get(url, proxies=proxy_dict).text

df = pd.read_html(io.StringIO(s))

df = df[0] #convert list to dataframe

**Belangrijkste variabelen**

Definieer het maximaal aantal kinderen per vierkante meter in Haarlem:

In [15]:
aantal_kinderen_per_m2 = 11

De totale oppervlakte in vierkante meter van de stad Haarlem, rekening houdend met het verschil tussen water en land:
<br>*Bron: Wikipedia*

In [16]:
oppervlakte_land = 29170000
oppervlakte_water = 2920000
oppervlakte_haarlem = oppervlakte_land + oppervlakte_water

Bereken het maximaal aantal kinderen dat in Haarlem past:

In [24]:
max_kinderen = oppervlakte_haarlem * aantal_kinderen_per_m2
max_kinderen

352990000

Waarvan op het land:

In [23]:
max_land = oppervlakte_land * aantal_kinderen_per_m2
max_land

320870000

En in het water:

In [25]:
max_water = oppervlakte_water * aantal_kinderen_per_m2
max_water

32120000

**Data pipeline** (korte methode)

In [19]:
def data_fixen(df): 
    return (df
            .copy()
            .rename(columns=dict(df.iloc[0]))                            # first row as header
            .iloc[1:]                                                    # remove obsolete header row
            .loc[lambda d: d['Land'] != 'Noord-Macedonië']               # corrupted count
            .assign(Inwonersaantal=lambda d: d['Inwonertal'] \
            .str.replace(".", "").str.split().str[0].astype(int))        # fix datatype and formatting
            .drop(columns=['Nr','Inwonertal'])                           # skip abundant info
           )

inwoners = data_fixen(df)

inwoners.head()

,Land,Inwonersaantal
1,China,1379302771
2,India,1281935911
3,Verenigde Staten,326625791
4,Indonesië,260580739
5,Brazilië,207353391


**Analyse vergelijking**

In [42]:
haarlem_df = inwoners.copy()
haarlem_df['haarlem'] = abs(haarlem_df['Inwonersaantal'] - max_kinderen)
haarlem_df['land'] = abs(haarlem_df['Inwonersaantal'] - max_land)
haarlem_df['water'] = abs(haarlem_df['Inwonersaantal'] - max_water)

Land waarvan het aantal inwoners nog het meeste lijkt op het maximum aantal kinderen dat in Haarlem past:

In [43]:
haarlem_df[haarlem_df['haarlem'] == min(abs(haarlem_df['Inwonersaantal'] - max_kinderen))][['Land', 'Inwonersaantal']]

,Land,Inwonersaantal
3,Verenigde Staten,326625791


Alleen op het land:

In [44]:
haarlem_df[haarlem_df['land'] == min(abs(haarlem_df['Inwonersaantal'] - max_land))][['Land', 'Inwonersaantal']]

,Land,Inwonersaantal
3,Verenigde Staten,326625791


En op het water:

In [46]:
haarlem_df[haarlem_df['water'] == min(abs(haarlem_df['Inwonersaantal'] - max_water))][['Land', 'Inwonersaantal']]

,Land,Inwonersaantal
41,Maleisië,31381992


En schrijf een nonsensverhaal voor Wateengast:

<< url naar wateengast.nl >>

**Toegift: een alternatieve datapipeline** (deze is beter schaalbaar als de boel wat complexer wordt)

In [47]:
def start_pipeline(df):
    return df.copy()

def fix_headers(df):
    return (df
           .rename(columns=dict(df.iloc[0])) # first row as header
           .iloc[1:] # remove obsolete header row
           )

def fix_rows(df):
    return (df
           .loc[lambda d: d['Land'] != 'Noord-Macedonië'] # corrupted count
           )
        
def fix_columns(df):
    return (df
           .assign(Inwonersaantal=lambda d: d['Inwonertal'].str.replace(".", "").str.split().str[0].astype(int))
           .drop(columns=['Nr','Inwonertal'])
           )

inwoners = (df
           .pipe(start_pipeline)
           .pipe(fix_headers)
           .pipe(fix_rows)
           .pipe(fix_columns)
           )

inwoners.head()

,Land,Inwonersaantal
1,China,1379302771
2,India,1281935911
3,Verenigde Staten,326625791
4,Indonesië,260580739
5,Brazilië,207353391
